Based on: https://github.dev/huggingface/notebooks/blob/main/sagemaker/23_stable_diffusion_inference/sagemaker-notebook.ipynb

TODO: Incooperate multi modal endpoint: 
- https://github.com/philschmid/huggingface-sagemaker-multi-container-endpoint/blob/master/sagemaker-notebook.ipynb
- https://github.com/vinayak-shanawad/AWS-SageMaker-Examples/blob/main/03_MultiModelEndpointWithHuggingFace/huggingface-sagemaker-multi-model-endpoint.ipynb

In [1]:
import sagemaker.huggingface
from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables from .env.

True

## Permissions

In [2]:
import sagemaker
import boto3

ACCESS_KEY = os.environ.get("AWS_SAGEMAKER_ACCESS_KEY")
SECRET_KEY = os.environ.get("AWS_SAGEMAKER_SECRET_ACCESS_KEY")
REGION = os.environ.get("AWS_BUCKET_REGION")

iam_client = boto3.client('iam', aws_access_key_id=ACCESS_KEY,
                          aws_secret_access_key=SECRET_KEY)

boto_session = boto3.Session(aws_access_key_id=ACCESS_KEY,
                             aws_secret_access_key=SECRET_KEY, region_name=REGION)


role = iam_client.get_role(
    RoleName='sagemaker')['Role']['Arn']
sess = sagemaker.Session(boto_session=boto_session)
region = sess.boto_region_name
sm_client = boto3.client('sagemaker')

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {region}")

sagemaker role arn: arn:aws:iam::254095540377:role/sagemaker
sagemaker bucket: sagemaker-eu-central-1-254095540377
sagemaker session region: eu-central-1


### Sagemaker Deep Learning Container

In [3]:
from sagemaker import image_uris

hf_inference_dlc = image_uris.retrieve(framework='huggingface', 
                                region=region, 
                                version='4.17', 
                                image_scope='inference', 
                                base_framework_version='pytorch1.10', 
                                py_version='py38', 
                                container_version='ubuntu20.04', 
                                instance_type='ml.g4dn.xlarge')

hf_inference_dlc

'763104351884.dkr.ecr.eu-central-1.amazonaws.com/huggingface-pytorch-inference:1.10-transformers4.17-gpu-py38-cu113-ubuntu20.04'

In [ ]:
# create SageMaker Model
image_uri = hf_inference_dlc
# image_uri = '763104351884.dkr.ecr.eu-central-1.amazonaws.com/huggingface-pytorch-inference:1.10-transformers4.17-gpu-py38-cu113-ubuntu20.04'

# Store training results
MODEL_BUCKET = "sagemaker-user-training"
multimodels_path = f's3://{MODEL_BUCKET}/finetuned_models'

deployment_name = "profaile-multi-model"

inferenceExecutionConfig = {"Mode": "Direct"}

primary_container = {
    'Image': image_uri,
    'Mode': 'MultiModel',
    'ModelDataUrl': multimodels_path,
    'Environment': {
        'SAGEMAKER_PROGRAM': 'inference.py',
        'SAGEMAKER_REGION': region,
        'SAGEMAKER_SUBMIT_DIRECTORY': multimodels_path
    }
}

create_model_response = sm_client.create_model(
    ModelName = deployment_name,
    InferenceExecutionConfig = inferenceExecutionConfig,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

# create SageMaker Endpoint configuration
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = f"{deployment_name}-epc",
    ProductionVariants=[
        {
        'InstanceType':'ml.g4dn.4xlarge',
        'InitialInstanceCount':1,
        'ModelName': deployment_name,
        'VariantName':'AllTraffic',
        'InitialVariantWeight':1
        }
    ])

print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

# create SageMaker Endpoint
endpoint_params = {
    'EndpointName': f"{deployment_name}-ep",
    'EndpointConfigName': f"{deployment_name}-epc",
}
endpoint_response = sm_client.create_endpoint(**endpoint_params)
print('EndpointArn = {}'.format(endpoint_response['EndpointArn']))

### Sending request to the model

In [ ]:
from PIL import Image
from io import BytesIO
from IPython.display import display
import base64
import matplotlib.pyplot as plt
import io
import requests

# helper decoder
def decode_base64_image(image_string):
  base64_image = base64.b64decode(image_string)
  buffer = BytesIO(base64_image)
  return Image.open(buffer)

def encode_base64_image(image):
  buffered = BytesIO()
  image.save(buffered, format="JPEG")
  return base64.b64encode(buffered.getvalue()).decode()

# display PIL images as grid
def display_images(images=None,columns=3, width=100, height=100):
    plt.figure(figsize=(width, height))
    for i, image in enumerate(images):
        plt.subplot(int(len(images) / columns + 1), columns, i + 1)
        plt.axis('off')
        plt.imshow(image)

def image_from_s3(bucket, key, s3):
  bucket = s3.Bucket(bucket)
  image = bucket.Object(key)
  img_data = image.get().get('Body').read()
  return Image.open(io.BytesIO(img_data))

# Get reference image for depth2image from S3 for user
USER_ID = "testing-test"

# Store instance images as train images and class images as test images
IMAGES_BUCKET = "sagemaker-user-io"
instance_input_path = f's3://{IMAGES_BUCKET}/{USER_ID}/'

# TODO: Path to example application picture which is used as starting point for inference.
# Will be stored somewhere in f's3://{IMAGES_BUCKET}/{USER_ID}/...'
instance_input_key = f"{USER_ID}/140e9145-5d97-4796-a94c-24283960d800.JPG"
s3 = boto3.resource('s3', aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY)
# Download the image
instance_input_image = image_from_s3(IMAGES_BUCKET, instance_input_key, s3)

url = "https://ca.slack-edge.com/TUTJQ5RNV-U02P1ET637V-7d91d73a4fd4-512"
instance_input_image = Image.open(requests.get(url, stream=True).raw)
display(instance_input_image)

In [ ]:
pos_prompt = "picture of qonvhs woman in a Business Jacket, perfect eyes, front view, closeup, centered frame, symmetric, studio lighting, clear and realistic face, uhd faces, pexels, 85mm, casual pose, 35mm film roll photo, hard light, detailed skin texture, masterpiece, sharp focus, pretty, lovely, adorable, attractive, hasselblad, candid street portrait"
neg_prompt = "blender, jewlery, chain,ugly, different eye color ,multiple hands, bad anatomy, bad proportions, unrealistic, full body, cropped, lowres, poorly drawn face, out of frame, poorly drawn hands, double, blurred, disfigured, deformed, repetitive, black and white"
init_image_base64 = encode_base64_image(instance_input_image)     # Has to be the image we use as reference. Encoded to base64 for request
strength = 0.825            # Regulates how "creative" the model is
num_images_per_prompt = 5  # How many images to generate per request
num_inference_steps = 50    # Depends on denoiser we choose. But in general 50 for DDIM is good
guidance_scale = 9.0        # How much the model follows the text prompt
SEED = 52362

import json
import boto3

invoke_client = boto3.client('sagemaker-runtime')

# Change timeout time
# https://stackoverflow.com/questions/31714800/boto3s-60s-timeout-on-swf
response = invoke_client.invoke_endpoint(
    EndpointName=f"{deployment_name}-ep",
    TargetModel=f'{USER_ID}.tar.gz',
    Body=json.dumps({
        # Check inference.py for parameters it expects
        "pos_prompt": pos_prompt,
        "neg_prompt": neg_prompt,
        "depth_image_base64": init_image_base64,
        "strength": strength,
        "num_images_per_prompt": num_images_per_prompt,
        "num_inference_steps": num_inference_steps,
        "guidance_scale": guidance_scale,
        "seed": SEED,
    }),
    Accept="application/json",
    ContentType="application/json",
    )

print(response)
response = response['Body'].read()

# TODO: Decode application image to base64 -> Allows to send the request to backend
decoded_images = [decode_base64_image(image) for image in response["generated_images"]]

# visualize generation
display_images(decoded_images)

### Util functions

In [23]:
from PIL import Image
from io import BytesIO
from IPython.display import display
import base64
import matplotlib.pyplot as plt
import io

# helper decoder
def decode_base64_image(image_string):
  base64_image = base64.b64decode(image_string)
  buffer = BytesIO(base64_image)
  return Image.open(buffer)

def encode_base64_image(image):
  buffered = BytesIO()
  image.save(buffered, format="JPEG")
  return base64.b64encode(buffered.getvalue()).decode()

# display PIL images as grid
def display_images(images=None,columns=3, width=100, height=100):
    plt.figure(figsize=(width, height))
    for i, image in enumerate(images):
        plt.subplot(int(len(images) / columns + 1), columns, i + 1)
        plt.axis('off')
        plt.imshow(image)

def image_from_s3(bucket, key, s3):
  bucket = s3.Bucket(bucket)
  image = bucket.Object(key)
  img_data = image.get().get('Body').read()
  return Image.open(io.BytesIO(img_data))

In [26]:
pos_prompt = "picture of qonvhs woman in a Business Jacket, perfect eyes, front view, closeup, centered frame, symmetric, studio lighting, clear and realistic face, uhd faces, pexels, 85mm, casual pose, 35mm film roll photo, hard light, detailed skin texture, masterpiece, sharp focus, pretty, lovely, adorable, attractive, hasselblad, candid street portrait"
neg_prompt = "blender, jewlery, chain,ugly, different eye color ,multiple hands, bad anatomy, bad proportions, unrealistic, full body, cropped, lowres, poorly drawn face, out of frame, poorly drawn hands, double, blurred, disfigured, deformed, repetitive, black and white"
init_image_base64 = encode_base64_image(instance_input_image)     # Has to be the image we use as reference. Encoded to base64 for request
strength = 0.825            # Regulates how "creative" the model is
num_images_per_prompt = 5  # How many images to generate per request
num_inference_steps = 50    # Depends on denoiser we choose. But in general 50 for DDIM is good
guidance_scale = 9.0        # How much the model follows the text prompt
SEED = 52362

# run prediction
response = predictor.predict(data={
    # Check inference.py for parameters it expects
    "pos_prompt": pos_prompt,
    "neg_prompt": neg_prompt,
    "depth_image_base64": init_image_base64,
    "strength": strength,
    "num_images_per_prompt": num_images_per_prompt,
    "num_inference_steps": num_inference_steps,
    "guidance_scale": guidance_scale,
    "seed": SEED,
  }
)

# TODO: Decode application image to base64 -> Allows to send the request to backend
decoded_images = [decode_base64_image(image) for image in response["generated_images"]]

# visualize generation
display_images(decoded_images)

In [27]:
predictor.delete_model()

In [28]:
predictor.delete_endpoint()